# auth.py Module

Handles authentication logic for admin and providers, including login, logout, password checks, and decorators.

In [ ]:
from werkzeug.security import check_password_hash, generate_password_hash
from flask import session, redirect, url_for

def login_required(f):
    def wrapper(*args, **kwargs):
        if not session.get('admin'):
            return redirect(url_for('admin_login'))
        return f(*args, **kwargs)
    return wrapper

def provider_required(f):
    def wrapper(*args, **kwargs):
        if not session.get('provider'):
            return redirect(url_for('provider_login'))
        return f(*args, **kwargs)
    return wrapper


# Authentication Module
All authentication, login, logout, password change, and session management logic from the Flask app.

In [ ]:
from flask import session, redirect, url_for, flash, request, render_template, abort
from functools import wraps
from werkzeug.security import check_password_hash, generate_password_hash
import datetime
from contextlib import closing

# --- Login Required Decorators ---
def login_required(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        # Uncomment to enable admin login check
        # if not session.get("admin"):
        #     return redirect(url_for("admin_login", next=request.path))
        return f(*args, **kwargs)
    return wrapper

def provider_required(f):
    @wraps(f)
    def wrapper(*args, **kwargs):
        if not session.get("provider"):
            return redirect(url_for("provider_login", next=request.path))
        return f(*args, **kwargs)
    return wrapper


In [ ]:
# --- Admin Authentication ---
def admin_login(app):
    if request.method == "POST":
        password = request.form.get("password", "")
        if check_password_hash(app.config["ADMIN_PASSWORD_HASH"], password):
            session["admin"] = True
            flash("Welcome back!", "success")
            return redirect(request.args.get("next") or url_for("admin_dashboard"))
        else:
            flash("Invalid password.", "error")
    return render_template("admin_login.html", title="Admin Login")

def admin_logout():
    session.pop("admin", None)
    flash("Logged out.", "info")
    return redirect(url_for("admin_login"))

def admin_change_password(app):
    if request.method == "POST":
        current = request.form.get("current", "")
        new = request.form.get("new", "")
        confirm = request.form.get("confirm", "")
        if not check_password_hash(app.config["ADMIN_PASSWORD_HASH"], current):
            flash("Current password is incorrect.", "error")
        elif not new or len(new) < 6:
            flash("New password must be at least 6 characters.", "error")
        elif new != confirm:
            flash("Passwords do not match.", "error")
        else:
            new_hash = generate_password_hash(new)
            with closing(get_db()) as db:
                cur = db.cursor()
                cur.execute("UPDATE auth SET admin_password_hash = ? WHERE id = 1", (new_hash,))
                db.commit()
            app.config["ADMIN_PASSWORD_HASH"] = new_hash
            flash("Admin password updated.", "success")
            return redirect(url_for("admin_dashboard"))
    return render_template("admin_change_password.html", title="Change Admin Password")


In [ ]:
# --- Provider Authentication ---
def provider_login(app):
    if request.method == "POST":
        email = request.form.get("email", "").strip().lower()
        password = request.form.get("password", "")
        if email and password:
            with closing(get_db()) as db:
                cur = db.cursor()
                cur.execute("SELECT id, password_hash, first_name, business_name FROM providers WHERE email = ? AND active = 1", (email,))
                provider = cur.fetchone()
                if provider and check_password_hash(provider["password_hash"], password):
                    session["provider"] = True
                    session["provider_id"] = provider["id"]
                    name = provider["first_name"] or provider["business_name"]
                    flash(f"Welcome back, {name}!", "success")
                    return redirect(request.args.get("next") or url_for("provider_dashboard"))
        if check_password_hash(app.config["PROVIDER_PASSWORD_HASH"], password):
            session["provider"] = True
            session["provider_id"] = 0
            flash("Welcome, Provider!", "success")
            return redirect(request.args.get("next") or url_for("provider_dashboard"))
        flash("Invalid email or password.", "error")
    return render_template("provider_login.html", title="Provider Login")

def provider_logout():
    session.pop("provider", None)
    flash("Logged out.", "info")
    return redirect(url_for("provider_login"))

def provider_change_password(app):
    if request.method == "POST":
        current = request.form.get("current", "")
        new = request.form.get("new", "")
        confirm = request.form.get("confirm", "")
        if not check_password_hash(app.config["PROVIDER_PASSWORD_HASH"], current):
            flash("Current password is incorrect.", "error")
        elif not new or len(new) < 6:
            flash("New password must be at least 6 characters.", "error")
        elif new != confirm:
            flash("Passwords do not match.", "error")
        else:
            new_hash = generate_password_hash(new)
            with closing(get_db()) as db:
                cur = db.cursor()
                cur.execute("UPDATE auth SET provider_password_hash = ? WHERE id = 1", (new_hash,))
                db.commit()
            app.config["PROVIDER_PASSWORD_HASH"] = new_hash
            flash("Provider password updated.", "success")
            return redirect(url_for("provider_dashboard"))
    return render_template("provider_change_password.html", title="Change Provider Password")


In [ ]:
# --- Provider Registration ---
def provider_register(app):
    if request.method == "POST":
        email = request.form.get("email", "").strip().lower()
        password = request.form.get("password", "").strip()
        confirm_password = request.form.get("confirm_password", "").strip()
        first_name = request.form.get("first_name", "").strip()
        business_name = request.form.get("business_name", "").strip() or "Independent"
        if not email or "@" not in email:
            flash("Please enter a valid email address.", "error")
            return render_template("provider_register.html", form=request.form, title="Join as Provider")
        if not password or len(password) < 6:
            flash("Password must be at least 6 characters.", "error")
            return render_template("provider_register.html", form=request.form, title="Join as Provider")
        if password != confirm_password:
            flash("Passwords do not match.", "error")
            return render_template("provider_register.html", form=request.form, title="Join as Provider")
        if not business_name:
            business_name = "Independent"
        with closing(get_db()) as db:
            cur = db.cursor()
            cur.execute("SELECT id FROM providers WHERE email = ?", (email,))
            if cur.fetchone():
                flash("An account with this email already exists.", "error")
                return render_template("provider_register.html", form=request.form, title="Join as Provider")
            password_hash = generate_password_hash(password)
            cur.execute(
                """
                INSERT INTO providers (email, password_hash, first_name, business_name, created_at)
                VALUES (?, ?, ?, ?, ?)
                """,
                (email, password_hash, first_name, business_name, datetime.datetime.utcnow().isoformat())
            )
            db.commit()
            provider_id = cur.lastrowid
        session["provider"] = True
        session["provider_id"] = provider_id
        flash(f"Welcome {first_name or business_name}! Your provider account has been created.", "success")
        return redirect(url_for("provider_dashboard"))
    return render_template("provider_register.html", form={}, title="Join as Provider")
